In [1]:
import os
import sys
from collections import defaultdict

import pydicom
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy.ndimage.morphology import binary_closing, binary_erosion  # Morphological operator

MRI_FRAMES = 50
MRI_MIN_RADIUS = 2
MRI_MAX_MYOCARDIUM = 20
MRI_BIG_RADIUS_FACTOR = 0.9
MRI_SMALL_RADIUS_FACTOR = 0.19
MRI_SEGMENTED_CHANNEL_MAP = {'background': 0, 'ventricle': 1, 'myocardium': 2}

In [2]:
!mkdir ./dcm_scratch
!rm ./dcm_scratch/*
!cp /mnt/ml4cvd/projects/bulk/cardiac_mri/1000387_20209_2_0.zip ./dcm_scratch/
!unzip ./dcm_scratch/1000387_20209_2_0.zip -d ./dcm_scratch/

mkdir: cannot create directory './dcm_scratch': File exists
Archive:  ./dcm_scratch/1000387_20209_2_0.zip
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512869723579391.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512875285379399.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512989554179454.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512840686479375.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512872296479395.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512960981879439.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512995575879460.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512840153979374.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512843747479382.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217512844179879383.dcm  
  inflating: ./dcm_scrat

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521944210179796.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521977850679813.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217522040471679846.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217522064880779854.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217522093739579868.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521918243079784.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521943325879793.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521974181379809.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121752202550879821.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521915563479781.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217521917446379783.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531745666981522.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531750534581552.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753175067781309.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531757081981592.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531536523980449.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531544687180501.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531545923580509.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531548895680525.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531560784180585.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531566114480611.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531567744780619.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531685514081205.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531688558181221.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531713028581322.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531719907081364.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531727041681408.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531727372881410.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531749557481546.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531751189581556.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531752183981562.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531756430481588.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531534036880433.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531590382580731.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531611039980833.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531628631880919.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753164534280801.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531653183581039.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531675195381151.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531677119181161.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531677501981163.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531688186081219.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531691172081235.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531720555681368.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2

...

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753175820781313.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531531861880419.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531552536780543.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531595630280757.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531599684380777.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531611853580837.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753161706980787.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531626163580907.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531641026280979.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531645537281001.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531661155581079.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753166550680811.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531679408981173.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.20180112175316898480783.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531693800781249.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121753171311981289.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531721527581374.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531723476381386.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531729970181426.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531734531081454.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531739778481486.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217531740753081492.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492435773878464.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492272956078379.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492392576978443.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492418522178453.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492273702878380.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492328161778403.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492335153778414.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492330367278407.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.20180112174923464878392.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492366072178430.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217492274725878381.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501344761978828.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501373313378841.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501427818878867.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501428262078868.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121750145529478860.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501259033578782.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501310980978804.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501313707678810.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501340442778819.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501346260278830.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217501375140578844.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2

  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510389218079203.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510421434479223.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510332114279179.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510429871679232.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510482287679256.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.201801121751059721579268.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510366457379200.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510395012679213.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510362322279194.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510454224779243.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2.18.141243.2018011217510482676679257.dcm  
  inflating: ./dcm_scratch/1.3.12.2.1107.5.2

In [3]:
dcm_dir = './dcm_scratch/'
series = defaultdict(list)
for dcm_file in os.listdir(dcm_dir):
    if not dcm_file.endswith('.dcm'):
        continue
    dcm = pydicom.read_file(dcm_dir + dcm_file)
    if 'cine_segmented_sax_inlinevf' == dcm.SeriesDescription.lower():
        cur_angle = (dcm.InstanceNumber - 1) // MRI_FRAMES 
        series[cur_angle].append(dcm)
print('len is:' , len(series))
for k in series:
    print(f'b series {k} has {len(series[k])} instances')

len is: 12
b series 7 has 50 instances
b series 4 has 50 instances
b series 11 has 50 instances
b series 6 has 50 instances
b series 3 has 50 instances
b series 5 has 50 instances
b series 0 has 50 instances
b series 10 has 50 instances
b series 8 has 50 instances
b series 1 has 50 instances
b series 9 has 50 instances
b series 2 has 50 instances


In [ ]:
MRI_MIN_RADIUS = 2
MRI_MAX_MYOCARDIUM = 20
MRI_BIG_RADIUS_FACTOR = 0.9
MRI_SMALL_RADIUS_FACTOR = 0.19
MRI_SEGMENTED_CHANNEL_MAP = {'background': 0, 'ventricle': 1, 'myocardium': 2}
def _is_mitral_valve_segmentation(d) -> bool:
    return d.SliceThickness == 6

def _get_overlay_from_dicom(d, debug=False):
    """Get an overlay from a DICOM file

    Morphological operators are used to transform the pixel outline of the myocardium
    to the labeled pixel masks for myocardium and left ventricle

    Arguments
        d: the dicom file
        stats: Counter to keep track of summary statistics

    Returns
        Tuple of two numpy arrays.
        The first is the raw overlay array with myocardium outline,
        The second is a pixel mask with 0 for background 1 for myocardium and 2 for ventricle
    """
    i_overlay = 0
    dicom_tag = 0x6000 + 2 * i_overlay
    overlay_raw = d[dicom_tag, 0x3000].value
    rows = d[dicom_tag, 0x0010].value  # rows = 512
    cols = d[dicom_tag, 0x0011].value  # cols = 512
    overlay_frames = d[dicom_tag, 0x0015].value
    bits_allocated = d[dicom_tag, 0x0100].value

    np_dtype = np.dtype('uint8')
    length_of_pixel_array = len(overlay_raw)
    expected_length = rows * cols
    if bits_allocated == 1:
        expected_bit_length = expected_length
        bit = 0
        overlay = np.ndarray(shape=(length_of_pixel_array * 8), dtype=np_dtype)
        for byte in overlay_raw:
            for bit in range(bit, bit + 8):
                overlay[bit] = byte & 0b1
                byte >>= 1
            bit += 1
        overlay = overlay[:expected_bit_length]
    if overlay_frames == 1:
        overlay = overlay.reshape(rows, cols)
        idx = np.where(overlay == 1)
        min_pos = (np.min(idx[0]), np.min(idx[1]))
        max_pos = (np.max(idx[0]), np.max(idx[1]))
        short_side = min((max_pos[0] - min_pos[0]), (max_pos[1] - min_pos[1]))
        small_radius = max(MRI_MIN_RADIUS, short_side * MRI_SMALL_RADIUS_FACTOR)
        big_radius = max(MRI_MIN_RADIUS+1, short_side * MRI_BIG_RADIUS_FACTOR)
        small_structure = _unit_disk(small_radius)
        m1 = binary_closing(overlay, small_structure).astype(np.int)
        big_structure = _unit_disk(big_radius)
        m2 = binary_closing(overlay, big_structure).astype(np.int)
        anatomical_mask = m1 + m2
        ventricle_pixels = np.count_nonzero(anatomical_mask == MRI_SEGMENTED_CHANNEL_MAP['ventricle'])
        myocardium_pixels = np.count_nonzero(anatomical_mask == MRI_SEGMENTED_CHANNEL_MAP['myocardium'])
        if ventricle_pixels == 0 and myocardium_pixels > MRI_MAX_MYOCARDIUM:
            erode_structure = _unit_disk(small_radius*1.5)
            anatomical_mask = anatomical_mask - binary_erosion(m1, erode_structure).astype(np.int)
            ventricle_pixels = np.count_nonzero(anatomical_mask == MRI_SEGMENTED_CHANNEL_MAP['ventricle'])
            print(f"rescue ventricle_pixels {ventricle_pixels} myo pixels: {myocardium_pixels} ")
        return overlay, anatomical_mask, ventricle_pixels
    
def _unit_disk(r) -> np.ndarray:
    y, x = np.ogrid[-r: r + 1, -r: r + 1]
    return (x ** 2 + y ** 2 <= r ** 2).astype(np.int)


def _outline_to_mask(labeled_outline, idx) -> np.ndarray:
    idx = np.where(labeled_outline == idx)
    poly = list(zip(idx[1].tolist(), idx[0].tolist()))
    img = Image.new("L", [labeled_outline.shape[1], labeled_outline.shape[0]], 0)
    ImageDraw.Draw(img).polygon(poly, outline=1, fill=1)
    return np.array(img)

In [ ]:
def plot_b_series(b_series, sides=7):
    _, axes = plt.subplots(sides, sides, figsize=(18, 24))
    for dcm in b_series:
        idx = (dcm.InstanceNumber-1)%50
        if idx >= sides*sides:
            continue
        if _is_mitral_valve_segmentation(dcm):
            axes[idx%sides, idx//sides].imshow(dcm.pixel_array)
        else:
            try:
                overlay, anatomical_mask, ventricle_pixels = _get_overlay_from_dicom(dcm)
                axes[idx%sides, idx//sides].imshow(np.ma.masked_where(anatomical_mask == 2, dcm.pixel_array))
            except KeyError:
                print(f'Could not get overlay at {dcm.InstanceNumber}, angle {s}')
                axes[idx, idx//sides].imshow(dcm.pixel_array)
        axes[idx%sides, idx//sides].set_yticklabels([])
        axes[idx%sides, idx//sides].set_xticklabels([])

In [ ]:
plot_b_series(series[2], sides=7)

In [ ]:
plot_b_series(series[8], sides=7)

In [ ]:
plot_b_series(series[5], sides=7)

In [ ]:
systoles = {}
diastoles = {}
systoles_pix = {}
diastoles_pix = {}
_, axes = plt.subplots(50, 12, figsize=(12, 36))
for s in series:
    for dcm in series[s]:
        if _is_mitral_valve_segmentation(dcm):
            axes[(dcm.InstanceNumber-1)%50, s].imshow(dcm.pixel_array)
            continue
        try:
            overlay, anatomical_mask, ventricle_pixels = _get_overlay_from_dicom(dcm)
            axes[(dcm.InstanceNumber-1)%50, s].imshow(np.ma.masked_where(anatomical_mask == 2, dcm.pixel_array))
            axes[(dcm.InstanceNumber-1)%50, s].set_yticklabels([])
            axes[(dcm.InstanceNumber-1)%50, s].set_xticklabels([])
        except KeyError:
            print(f'could get overlay at {dcm.InstanceNumber}, angle {s}')
            axes[(dcm.InstanceNumber-1)%50, s].imshow(dcm.pixel_array)
        if s not in diastoles:
            diastoles[s] = dcm
            diastoles_pix[s] = ventricle_pixels
            systoles[s] = dcm
            systoles_pix[s] = ventricle_pixels
        else:
            if ventricle_pixels > diastoles_pix[s]:
                diastoles[s] = dcm
                diastoles_pix[s] = ventricle_pixels
            if ventricle_pixels < systoles_pix[s]:
                systoles[s] = dcm
                systoles_pix[s] = ventricle_pixels

for angle in diastoles:
    print(f'Found systole at instance {systoles[angle].InstanceNumber}  pix: {systoles_pix[angle]}')
    print(f'Found diastole at instance {diastoles[angle].InstanceNumber}   pix: {diastoles_pix[angle]}\n')

In [ ]:
print (series.keys())